Test framework

In [ ]:

from imports import *

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from networkx.algorithms import community  

import time
import config
from utiles import record_memory_usage
from imports import * 

# ─────────────────────────────────────────────────────────────────────────────
# 0) Reiniciar 
# ─────────────────────────────────────────────────────────────────────────────
config.memory_times  = []
config.memory_values = []
config.start_time    = time.time()

# ─────────────────────────────────────────────────────────────────────────────
# 1)  dataframe
# ─────────────────────────────────────────────────────────────────────────────
df = pd.read_csv("../IoT Network Intrusion Dataset.csv")
record_memory_usage(label="after_load", record_every=1, iteration=0)

df = df.drop(columns=[
    'Flow_ID', 'Cat', 'Sub_Cat', 'Timestamp',
    'Src_IP', 'Dst_IP', 'Label'
])
record_memory_usage(label="after_drop_cols", record_every=1, iteration=1)

df_sample = df.sample(n=70000, random_state=42)
df_sample.replace([np.inf, -np.inf], np.nan, inplace=True)
df_sample.dropna(inplace=True)
record_memory_usage(label="after_sample_clean", record_every=1, iteration=2)

# ─────────────────────────────────────────────────────────────────────────────
# 2) Transposición y similitud
# ─────────────────────────────────────────────────────────────────────────────
df_transposed = df_sample.transpose()
record_memory_usage(label="after_transpose", record_every=1, iteration=3)

similarity_matrix = cosine_similarity(df_transposed)
record_memory_usage(label="after_cosine", record_every=1, iteration=4)

# ─────────────────────────────────────────────────────────────────────────────
# 3) grafo 
# ─────────────────────────────────────────────────────────────────────────────
threshold = 0.99
features  = list(df_transposed.index)

G = nx.Graph()
G.add_nodes_from(features)
record_memory_usage(label="after_add_nodes", record_every=1, iteration=5)

for i in range(len(features)):
    for j in range(i + 1, len(features)):
        if similarity_matrix[i, j] >= threshold:
            G.add_edge(features[i], features[j], weight=similarity_matrix[i, j])
    record_memory_usage(label="edge_build", record_every=500, iteration=i)

t_fs_end = time.time() - config.start_time
print(f"Feature selection done in {t_fs_end:.1f}s")

# ─────────────────────────────────────────────────────────────────────────────
# 4) comunidades
# ─────────────────────────────────────────────────────────────────────────────
communities = community.greedy_modularity_communities(G)
record_memory_usage(label="after_communities", record_every=1, iteration=0)

selected_features = []
for idx, comm in enumerate(communities):
    if len(comm) > 1:
        subg = G.subgraph(comm)
        cent = nx.degree_centrality(subg)
        rep = max(cent, key=cent.get)
        selected_features.append(rep)
    else:
        selected_features.extend(comm)
    record_memory_usage(label="centrality", record_every=5, iteration=idx)

t_fs_end2 = time.time() - config.start_time
print(f"Community & repr. selection done in {t_fs_end2:.1f}s")

# ─────────────────────────────────────────────────────────────────────────────
# 5) plot
# ─────────────────────────────────────────────────────────────────────────────
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G)
colors = ["red" if n in selected_features else "skyblue" for n in G]
nx.draw_networkx(
    G, pos,
    node_color=colors, node_size=50,
    with_labels=False, edge_color="gray"
)
plt.title("Graph-Based Feature Selection")
plt.axis("off")
record_memory_usage(label="after_plot", record_every=1, iteration=0)

# ─────────────────────────────────────────────────────────────────────────────
# 6) DataFrame final
# ─────────────────────────────────────────────────────────────────────────────
all_feats       = set(G.nodes())
unselected      = all_feats - set(selected_features)
df_final        = pd.read_csv("../IoT Network Intrusion Dataset.csv")
df_final        = df_final.drop(columns=list(unselected))
record_memory_usage(label="after_final_drop", record_every=1, iteration=0)

t_fs_complete = time.time() - config.start_time
print(f"Total feature selection phase in {t_fs_complete:.1f}s")


In [12]:
import optuna
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import matthews_corrcoef
from data import *
from dataset import *
from neural import *
from symbolic import *
from trainer import *

df=df_final
#df= df[0:100]
time.sleep(0.1) 
t_fs_complete = time.time() - config.start_time

# ————————————————————————————————————————————————————————————————
# --- FASE 1: Crear grafos directos ---------------
graphs_info = create_graphs_from_dataset(df, record_every=100)
t_directos   = time.time() - config.start_time

# ————————————————————————————————————————————————————————————————
# --- FASE 2:  estadisticos de los grafos ---
results = analyze_graphs(graphs_info,
                         output_filename=config.grafos_directos2,
                         record_every=100)
t_analysis   = time.time() - config.start_time

# ————————————————————————————————————————————————————————————————
# --- FASE 3: recíprocos --------
convert_and_save_line_graphs(config.grafos_directos2,
                             config.grafos_reciprocos3,
                             record_every=100)
t_recip      = time.time() - config.start_time

# ————————————————————————————————————————————————————————————————
# --- FASE 4: Extraer MST --------------------------
compute_and_save_mst_graphs(config.grafos_reciprocos3,
                            config.grafos_mst,
                            record_every=50)
t_mst        = time.time() - config.start_time


# ————————————————————————————————————————————————————————————————
# ————————————————————————————————————————————————————————————————
# ————————————————————————————————————————————————————————————————
# --- FASE 5: Extraer MST - Optuna hyperparameter search
t_opt_start    = time.time() - config.start_time

def objective(trial: optuna.Trial) -> float:
    h_dim      = trial.suggest_categorical("h_dim",      [32, 64, 128])
    z_dim      = trial.suggest_categorical("z_dim",      [16, 32, 64])
    n_layers   = trial.suggest_int        ("n_layers",   2, 4)
    dropout    = trial.suggest_float      ("dropout",    0.1, 0.5)
    act_fn     = trial.suggest_categorical("act_fn",     ["relu", "tanh", "leakyrelu"])
    lr         = trial.suggest_float      ("lr",         1e-4, 1e-2, log=True)
    beta_KL    = trial.suggest_float      ("beta_KL",    1e-3, 1.0, log=True)
    gamma_CE   = trial.suggest_float      ("gamma_CE",   1e-3, 1.0, log=True)
    lam_rec    = trial.suggest_float      ("lam_rec",    0.0, 1.0)
    lam_lat    = trial.suggest_float      ("lam_lat",    0.0, 1.0 - lam_rec)
    eta_fuse   = trial.suggest_float      ("eta_fuse",   0.0, 1.0)
    k_rule     = trial.suggest_int        ("k_rule",     1, 3)
    tau        = trial.suggest_float      ("tau",        0.0, 1.0)

    train_raw, test_raw, val_raw = partition_graphs(
    pkl_file    = r"\grafos\grafos_mst.pkl",
    train       = 200,
    test        = 2000,
    validacion  = 1400,
    min_normals_test=58,
    min_normals_valid=40)

    rule_stats = compute_descriptors_statistics(train_raw)
    scaler     = fit_feature_scaler(train_raw)

    train_ds = GraphDataset(
        train_raw, scaler=scaler,
        rule_stats=rule_stats,
        k_sigma=k_rule, tau=tau
    )
    val_ds = GraphDataset(
        val_raw, scaler=scaler,
        rule_stats=rule_stats,
        k_sigma=k_rule, tau=tau
    )
    train_loader = DataLoader(
        train_ds, batch_size=16, shuffle=True,
        collate_fn=collate_small_graphs
    )
    val_loader = DataLoader(
        val_ds,   batch_size=16, shuffle=False,
        collate_fn=collate_small_graphs
    )

   
    in_dim    = train_ds[0][1].shape[1]
    model     = VGAE(
        in_dim      = in_dim,
        hidden_dim  = h_dim,
        latent_dim  = z_dim,
        num_layers  = n_layers,
        activation  = act_fn,
        dropout     = dropout,
    )
    optimizer = Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=5)
    early_st  = EarlyStop(patience=10, delta=1e-4)

    trainer = Trainer(
        model         = model,
        loaders       = (train_loader, val_loader),
        optimizer     = optimizer,
        scheduler     = scheduler,
        early_stopper = early_st,
        beta_KL       = beta_KL,
        gamma_CE      = gamma_CE,
        lambdas       = (lam_rec, lam_lat),
        eta_fuse      = eta_fuse,
        device        = "cpu",
    )

    best_val_mcc = trainer.fit(epochs=50, trial=trial)
    return best_val_mcc


sampler = optuna.samplers.TPESampler(seed=42)
pruner  = optuna.pruners.MedianPruner(n_warmup_steps=5)
study   = optuna.create_study(direction="maximize",
                              sampler=sampler,
                              pruner=pruner)
study.optimize(objective, n_trials=40)

t_opt_end      = time.time() - config.start_time

# ————————————————————————————————————————————————————————————————
best = study.best_params

train_raw, test_raw, val_raw = partition_graphs(
    pkl_file          = r"\grafos\grafos_mst.pkl",
    train             = 200,
    test              = 1500,
    validacion        = 1500,
    min_normals_test  = 68,
    min_normals_valid = 30,
)

rule_stats = compute_descriptors_statistics(train_raw)
scaler     = fit_feature_scaler(train_raw)

trainval_raw     = train_raw + val_raw
trainval_ds      = GraphDataset(trainval_raw, scaler, rule_stats,
                                k_sigma=best["k_rule"], tau=best["tau"])
test_ds          = GraphDataset(test_raw,    scaler, rule_stats,
                                k_sigma=best["k_rule"], tau=best["tau"])

trainval_loader  = DataLoader(trainval_ds, batch_size=16, shuffle=True,
                              collate_fn=collate_small_graphs)
test_loader      = DataLoader(test_ds,     batch_size=16, shuffle=False,
                              collate_fn=collate_small_graphs)

in_dim    = trainval_ds[0][1].shape[1]
model     = VGAE(
    in_dim      = in_dim,
    hidden_dim  = best["h_dim"],
    latent_dim  = best["z_dim"],
    num_layers  = best["n_layers"],
    activation  = best["act_fn"],
    dropout     = best["dropout"],
)
optimizer = Adam(model.parameters(), lr=best["lr"])
scheduler = ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=5)
early_st  = EarlyStop(patience=10, delta=1e-4)

trainer = Trainer(
    model         = model,
    loaders       = (trainval_loader, test_loader),
    optimizer     = optimizer,
    scheduler     = scheduler,
    early_stopper = early_st,
    beta_KL       = best["beta_KL"],
    gamma_CE      = best["gamma_CE"],
    lambdas       = (best["lam_rec"], best["lam_lat"]),
    eta_fuse      = best["eta_fuse"],
    device        = "cpu",
)


start_final = time.time()
best_val_mcc = trainer.fit(epochs=20, trial=None)
t_final_train = time.time() - start_final

scores, latents, p_hats, s_syms, y_true = trainer.evaluate(test_loader)

s_neural = [
    best["lam_rec"] * rec
  + best["lam_lat"] * lat
  + (1 - best["lam_rec"] - best["lam_lat"]) * p
  for rec, lat, p in zip(scores, latents, p_hats)
]
s_final  = [
    best["eta_fuse"] * sn + (1 - best["eta_fuse"]) * ss
    for sn, ss in zip(s_neural, s_syms)
]

y_pred = [1 if sf >= trainer.best_thr else 0 for sf in s_final]

print(">>> TEST MCC:", matthews_corrcoef(y_true, y_pred))
print("\nClassification Report:\n",
      classification_report(y_true, y_pred,
                            labels=[0,1],
                            target_names=["Normal","Anomaly"],
                            digits=4))
print("\nConfusion Matrix:\n",
      confusion_matrix(y_true, y_pred, labels=[0,1]))

t_final = time.time() - config.start_time
print(f"\n(Fase 7 completada en {t_final:.1f}s desde el inicio)")



times = config.memory_times
mems  = config.memory_values

fig, ax = plt.subplots(figsize=(12,5))
ax.plot(times, mems, '-o', label='Memory (GB)')

ax.axvspan(0.0,           t_fs_complete, color='C0', alpha=0.3, label='0. Feature sel.')
ax.axvspan(t_fs_complete, t_directos,    color='C1', alpha=0.3, label='1. Direct graphs')
ax.axvspan(t_directos,    t_analysis,    color='C2', alpha=0.3, label='2. Analysis graphs')
ax.axvspan(t_analysis,    t_recip,       color='C3', alpha=0.3, label='3. Reciprocal graphs')
ax.axvspan(t_recip,       t_mst,         color='C4', alpha=0.3, label='4. MST extraction')
ax.axvspan(t_opt_start,   t_opt_end,     color='C5', alpha=0.3, label='5. Optuna search')
ax.axvspan(t_opt_end,     t_final,       color='C6', alpha=0.3, label='6. Final test')

ax.set_xlabel("Time (s)")
ax.set_ylabel("Memory (GB)")
ax.set_title("Memory vs time in the pipeline")
ax.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.tight_layout()
plt.show()


[I 2025-04-23 23:54:27,890] A new study created in memory with name: no-name-7d7d3b97-0d51-48dc-97c8-d8c96ba3bbb4
2025-04-23 23:54:27,893 - INFO - Loading graphs from \grafos\grafos_mst.pkl
2025-04-23 23:54:37,965 - INFO - Total loaded: 4304
2025-04-23 23:54:37,968 - INFO - Training set: 200 normals
2025-04-23 23:54:37,971 - INFO - Test set: 2000 graphs (58 normals)
2025-04-23 23:54:37,972 - INFO - Validation set: 1400 graphs (40 normals)


Graphs loaded from \grafos\grafos_mst.pkl


2025-04-23 23:54:41,331 - INFO - Partitioning done: train=200, test=2000, valid=1400
2025-04-23 23:54:41,352 - INFO - Descriptor 'num_nodes': mean = 4.0200, std = 1.9182
2025-04-23 23:54:41,353 - INFO - Descriptor 'num_edges': mean = 2.1650, std = 1.2197
2025-04-23 23:54:41,353 - INFO - Descriptor 'density': mean = 0.5315, std = 0.3388
2025-04-23 23:54:41,353 - INFO - Descriptor 'average_degree': mean = 1.0542, std = 0.1095
2025-04-23 23:54:41,354 - INFO - Descriptor 'average_clustering': mean = 0.0015, std = 0.0212
2025-04-23 23:54:41,354 - INFO - Descriptor 'diameter': mean = nan, std = nan
2025-04-23 23:54:41,355 - INFO - Descriptor 'average_shortest_path_length': mean = nan, std = nan
2025-04-23 23:54:41,355 - INFO - Descriptor 'spectral_gap': mean = 1.7748, std = 0.4271
2025-04-23 23:54:41,355 - INFO - Descriptor 'data_mean_Src_Port': mean = 15094.9909, std = 12951.6082
2025-04-23 23:54:41,356 - INFO - Descriptor 'data_mean_Dst_Port': mean = 42682.3978, std = 12974.1427
2025-04-23

Graphs loaded from \grafos\grafos_mst.pkl


2025-04-23 23:57:05,922 - INFO - Partitioning done: train=200, test=2000, valid=1400
2025-04-23 23:57:05,943 - INFO - Descriptor 'num_nodes': mean = 3.9550, std = 1.9552
2025-04-23 23:57:05,944 - INFO - Descriptor 'num_edges': mean = 2.1100, std = 1.2074
2025-04-23 23:57:05,944 - INFO - Descriptor 'density': mean = 0.5554, std = 0.3507
2025-04-23 23:57:05,944 - INFO - Descriptor 'average_degree': mean = 1.0464, std = 0.1043
2025-04-23 23:57:05,945 - INFO - Descriptor 'average_clustering': mean = 0.0000, std = 0.0000
2025-04-23 23:57:05,945 - INFO - Descriptor 'diameter': mean = nan, std = nan
2025-04-23 23:57:05,945 - INFO - Descriptor 'average_shortest_path_length': mean = nan, std = nan
2025-04-23 23:57:05,946 - INFO - Descriptor 'spectral_gap': mean = 1.8098, std = 0.4024
2025-04-23 23:57:05,946 - INFO - Descriptor 'data_mean_Src_Port': mean = 15131.5671, std = 12965.3056
2025-04-23 23:57:05,947 - INFO - Descriptor 'data_mean_Dst_Port': mean = 42644.8676, std = 12989.9480
2025-04-23

...

=== Resultados Optuna ===
Best M​CC:     0.961937531746931
Best params:  {'h_dim': 32, 'z_dim': 16, 'n_layers': 3, 'dropout': 0.2770713002465045, 'act_fn': 'relu', 'lr': 0.0037643083521274566, 'beta_KL': 0.04535094634779837, 'gamma_CE': 0.015043838978395768, 'lam_rec': 0.43729139171842235, 'lam_lat': 0.31045483654546724, 'eta_fuse': 0.028478291139971468, 'k_rule': 3, 'tau': 0.9168184602315099}
Mejores params: {'h_dim': 32, 'z_dim': 16, 'n_layers': 3, 'dropout': 0.2770713002465045, 'act_fn': 'relu', 'lr': 0.0037643083521274566, 'beta_KL': 0.04535094634779837, 'gamma_CE': 0.015043838978395768, 'lam_rec': 0.43729139171842235, 'lam_lat': 0.31045483654546724, 'eta_fuse': 0.028478291139971468, 'k_rule': 3, 'tau': 0.9168184602315099}


2025-04-24 00:35:18,756 - INFO - Total loaded: 4304
2025-04-24 00:35:18,759 - INFO - Training set: 200 normals
2025-04-24 00:35:18,762 - INFO - Test set: 1500 graphs (68 normals)
2025-04-24 00:35:18,763 - INFO - Validation set: 1500 graphs (30 normals)
2025-04-24 00:35:22,084 - INFO - Partitioning done: train=200, test=1500, valid=1500


Graphs loaded from \grafos\grafos_mst.pkl


2025-04-24 00:35:22,108 - INFO - Descriptor 'num_nodes': mean = 4.2150, std = 2.0122
2025-04-24 00:35:22,109 - INFO - Descriptor 'num_edges': mean = 2.2700, std = 1.2717
2025-04-24 00:35:22,110 - INFO - Descriptor 'density': mean = 0.5122, std = 0.3434
2025-04-24 00:35:22,110 - INFO - Descriptor 'average_degree': mean = 1.0531, std = 0.1108
2025-04-24 00:35:22,110 - INFO - Descriptor 'average_clustering': mean = 0.0000, std = 0.0000
2025-04-24 00:35:22,111 - INFO - Descriptor 'diameter': mean = nan, std = nan
2025-04-24 00:35:22,111 - INFO - Descriptor 'average_shortest_path_length': mean = nan, std = nan
2025-04-24 00:35:22,111 - INFO - Descriptor 'spectral_gap': mean = 1.7797, std = 0.4332
2025-04-24 00:35:22,112 - INFO - Descriptor 'data_mean_Src_Port': mean = 14830.1290, std = 12364.5429
2025-04-24 00:35:22,112 - INFO - Descriptor 'data_mean_Dst_Port': mean = 42925.0088, std = 12381.6448
2025-04-24 00:35:22,112 - INFO - Descriptor 'data_mean_Protocol': mean = 6.0313, std = 0.1232
2

>>> TEST MCC: 0.9661389485159884

Classification Report:
               precision    recall  f1-score   support

      Normal     1.0000    0.9367    0.9673        79
     Anomaly     0.9965    1.0000    0.9982      1421

    accuracy                         0.9967      1500
   macro avg     0.9982    0.9684    0.9828      1500
weighted avg     0.9967    0.9967    0.9966      1500


Confusion Matrix:
 [[  74    5]
 [   0 1421]]
